<img src="Logo.png" width="100" align="left"/> 

# <center> Unit 3 Project </center>
#  <center> Third section : supervised task </center>

In this notebook you will be building and training a supervised learning model to classify your data.

For this task we will be using another classification model "The random forests" model.

Steps for this task: 
1. Load the already clustered dataset 
2. Take into consideration that in this task we will not be using the already added column "Cluster" 
3. Split your data.
3. Build your model using the SKlearn RandomForestClassifier class 
4. classify your data and test the performance of your model 
5. Evaluate the model ( accepted models should have at least an accuracy of 86%). Play with hyper parameters and provide a report about that.
6. Provide evidence on the quality of your model (not overfitted good metrics)
7. Create a new test dataset that contains the testset + an additional column called "predicted_class" stating the class predicted by your random forest classifier for each data point of the test set.

## 1. Load the data and split the data:

In [1]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd 
from sklearn.model_selection import train_test_split

In [2]:
# To-Do:  load the data 
df = pd.read_csv("outputs/unsupervised.csv")
df.head()

,ID,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT,cluster
0,1,0.0,32,1.0,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0,3
1,2,0.0,32,1.0,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5,3
2,3,0.0,32,1.0,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3,3
3,4,0.0,32,1.0,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7,3
4,5,0.0,32,1.0,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7,3


In [3]:
# To-Do : keep only the columns to be used : all features except ID, cluster 
# The target here is the Category column 
# Do not forget to split your data (this is a classification task)
# test set size should be 20% of the data 
X = df.drop(["ID","cluster"],axis=1).values
Y = df[["Category"]].values

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2,random_state=42)

## 2. Building the model and training and evaluate the performance: 

In [4]:
# To-do build the model and train it 
# note that you will be providing explanation about the hyper parameter tuning 
# So you will be iterating a number of times before getting the desired performance 
model = RandomForestClassifier()
model.fit(X_train,Y_train)

C:\Users\DELL\AppData\Local\Temp/ipykernel_22900/1925296566.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train,Y_train)


RandomForestClassifier()

In [20]:
y_hat = model.predict(X_test)

In [26]:
# To-do : evaluate the model in terms of accuracy and precision 
# Provide evidence that your model is not overfitting 
from sklearn.metrics import precision_score, accuracy_score

y_hat_train = model.predict(X_train)

print(f"Accuracy on Training set : {accuracy_score(Y_train,y_hat_train)}")
print(f"Precision on Training set : {precision_score(Y_train,y_hat_train,average='weighted')}")

print(f"Accuracy on Test set : {accuracy_score(Y_test,y_hat)} ")
print(f"Precision on Test set : {precision_score(Y_test,y_hat,average='weighted')}")

Accuracy on Training set : 1.0
Precision on Training set : 1.0
Accuracy on Test set : 0.9512195121951219 
Precision on Test set : 0.9574008884418741


#### Hyper-parameter Tuning using GridSearch

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [14]:
def gridSearch(X_train,Y_train,X_test,Y_test,tuned_parameters,metric="precision"):
    print(f'Fine-Tuning parameter for metric : {metric}')
    
    clf = GridSearchCV(RandomForestClassifier(),param_grid=tuned_parameters,scoring='%s_macro' % metric,verbose=5)
    results = clf.fit(X_train,Y_train)
    
    print(f'Best score : {results.best_score_}, using parameters : {results.best_params_}')
    
    print(f'results : {clf.cv_results_}')
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    params = clf.cv_results_['params']

    for mean, stdev, param in zip(means, stds, params):
        print('{0} ({1}) with: {2}'.format(mean, stdev, param))
    
    print("Classification Report ")
    y_true,y_pred = Y_test,clf.predict(X_test)
    
    print(classification_report(y_true,y_pred))

In [17]:
tuned_params = [{
        #"n_estimators":range(50,101),
        "criterion": ["gini", "entropy"],
        #"max_depth": range(5,11),
        #"min_samples_split":range(2,6),
        #"min_samples_leaf":range(1,4),
        "max_features":["sqrt","log2"]
                     }]
#scores = ["precision","accuracy"]

In [18]:
import warnings

warnings.filterwarnings("ignore")

gridSearch(X_train,Y_train,X_test,Y_test,tuned_params,scores)

Fine-Tuning parameter for metric : ['precision', 'accuracy']


ValueError: "['precision', 'accuracy']_macro" is not a valid scoring value. Use sorted(sklearn.metrics.SCORERS.keys()) to get valid options.

In [25]:
from sklearn.metrics import classification_report

print(f'Classification report on test set : \n{classification_report(Y_test,y_hat)}')

Classification report on test set : 
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99        96
         1.0       1.00      0.33      0.50         3
         2.0       0.75      1.00      0.86         9
         3.0       0.67      0.67      0.67         6
         4.0       1.00      0.78      0.88         9

    accuracy                           0.95       123
   macro avg       0.88      0.76      0.78       123
weighted avg       0.96      0.95      0.95       123



> Hint : A Perfect accuracy on the train set suggest that we have an overfitted model So the student should be able to provide a detailed table about the hyper parameters / parameters tuning with a good conclusion stating that the model has at least an accuracy of 86% on the test set without signs of overfitting  

## 3. Create the summary test set with the additional predicted class column: 
In this part you need to add the predicted class as a column to your test dataframe and save this one 

In [ ]:
# To-Do : create the complete test dataframe : it should contain all the feature column + the actual target and the ID as well  
test_df["Category"] = 

/home/soumia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
test_df.head()

,ID,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT,cluster,Category
248,249,55,0,28.1,65.5,16.6,17.5,2.8,5.58,4.39,65.0,26.2,62.4,3,0
365,366,39,1,31.4,106.0,16.6,17.0,2.4,5.95,5.30,68.0,22.9,72.3,3,0
432,433,48,1,43.7,50.1,17.3,26.3,8.1,8.15,5.38,64.0,13.4,73.1,3,0
610,611,62,1,32.0,416.6,5.9,110.3,50.0,5.57,6.30,55.7,650.9,68.5,2,3
132,133,44,0,35.5,81.7,27.5,29.5,6.4,8.81,6.65,83.0,24.1,68.0,3,0


In [ ]:
# To-Do : Add the predicted_class column 
test_df["Predicted_class"] = 

/home/soumia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
test_df.head()

,ID,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT,cluster,Category,Predicted_class
248,249,55,0,28.1,65.5,16.6,17.5,2.8,5.58,4.39,65.0,26.2,62.4,3,0,0
365,366,39,1,31.4,106.0,16.6,17.0,2.4,5.95,5.30,68.0,22.9,72.3,3,0,0
432,433,48,1,43.7,50.1,17.3,26.3,8.1,8.15,5.38,64.0,13.4,73.1,3,0,0
610,611,62,1,32.0,416.6,5.9,110.3,50.0,5.57,6.30,55.7,650.9,68.5,2,3,3
132,133,44,0,35.5,81.7,27.5,29.5,6.4,8.81,6.65,83.0,24.1,68.0,3,0,0


> Make sure you have 16 column in this test set  

In [ ]:
# Save the test set 
test_df.to_csv("test_summary.csv")